In [1]:
import pickle
from glob import glob

files = sorted(glob('/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/*.pkl'))
len(files)

13418

In [2]:
from datetime import datetime, timedelta
import pathlib

atleast = datetime(2023, 6, 10)

filtered = []
for f in files:
    fname = pathlib.Path(f)
    if datetime.fromtimestamp(fname.stat().st_mtime) > atleast:
        filtered.append(f)
        
files = sorted(filtered)
len(files)

5795

In [3]:
from datasets import Audio
from tqdm import tqdm
import numpy as np
import IPython.display as ipd

In [4]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [5]:
with open(files[0], 'rb') as fopen:
    data = pickle.load(fopen)

In [6]:
max(data[0]['asr_model'][0], key=data[0]['asr_model'][0].get)

'ms'

In [7]:
data[0]

{'wav_data': '/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/"Ada_harta_baru_betul_duda_hot"_-_Hasnol_takut_digelar_dayus,_sedia_kembali_kepada_bekas_isteri_[3TYBFOpIHbc]/0.mp3',
 'start': 0.0,
 'asr_model': ({'sl': 2.1883515728404745e-05,
   'hr': 7.0581018007942475e-06,
   'bn': 5.781698928331025e-05,
   'et': 4.332884600444231e-06,
   'yo': 0.0011710386024788022,
   'kn': 2.853538899216801e-05,
   'mr': 3.670094156404957e-05,
   'pt': 6.328371091512963e-05,
   'ms': 0.945112407207489,
   'sd': 0.00011648763029370457,
   'da': 0.00022656652436126024,
   'ja': 0.00048791812150739133,
   'oc': 0.00013697757094632834,
   'de': 0.0015614362200722098,
   'sw': 0.000652390590403229,
   'sq': 1.6484786101500504e-05,
   'yi': 5.956028871878516e-06,
   'fi': 2.3270855308510363e-05,
   'ml': 2.6278255973011255e-05,
   'ru': 0.0002895639627240598,
   'lt': 8.882307156454772e-06,
   'si': 0.00020464169210754335,
   'te': 2.0325713194324635e-05,
   'th': 0.0002541006251703948,
   'sr': 

In [8]:
ms_score = [s['avg_logprob'] for s in data[0]['asr_model'][1]['segments']]
en_score = [s['avg_logprob'] for s in data[0]['asr_model'][2]['segments']]

if np.mean(ms_score) > np.mean(en_score):
    selected = data[0]['asr_model'][1]['segments']
else:
    selected = data[0]['asr_model'][2]['segments']

In [9]:
i = 0

s = selected[i]['start']
e = selected[i]['end']
t = selected[i]['text']
s, e, t

(0.0, 2.5, ' Bercinta tu senang, kahwin pun senang')

In [10]:
f = data[0]['wav_data']
audio = reader.decode_example(reader.encode_example(f))['array']
y = audio[int(s * sr): int(e * sr)]

In [11]:
ipd.Audio(y, rate = sr)

In [12]:
import malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [13]:
vad = malaya_speech.vad.webrtc()

In [14]:
y_int = malaya_speech.astype.float_to_int(y)
frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr))
np.mean([vad(frame) for frame in frames_int])

0.9761904761904762

In [15]:
import malaya
from malaya.text.normalization import cardinal

tokenizer = malaya.tokenizer.Tokenizer(hypen = False, parliament = False, time = False, time_pukul = False,
                                      temperature = False, distance = False, volume = False, duration = False,
                                      weight = False, date = False, money = False)

In [16]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    tokenized = tokenizer.tokenize(string)
    string = ' '.join(tokenized)
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [17]:
directory = '/home/husein/ssd3/stt/whisper-v5'

In [18]:
# !rm -rf {directory}
!mkdir {directory}

mkdir: cannot create directory ‘/home/husein/ssd3/stt/whisper-v5’: File exists


In [19]:
import torchaudio
import torch
import os

In [20]:
dataset = []
index = 0
for no, f in enumerate(files):
    
    with open(f, 'rb') as fopen:
        data = pickle.load(fopen)
    
    for i in tqdm(range(len(data))):
        lang = max(data[i]['asr_model'][0], key=data[i]['asr_model'][0].get)
        if lang not in {'ms', 'en'}:
            continue
        
        ms_score = [s['avg_logprob'] for s in data[i]['asr_model'][1]['segments']]
        en_score = [s['avg_logprob'] for s in data[i]['asr_model'][2]['segments']]

        if np.mean(ms_score) > np.mean(en_score):
            selected = data[i]['asr_model'][1]['segments']
        else:
            selected = data[i]['asr_model'][2]['segments']
            
        audio = reader.decode_example(reader.encode_example(data[i]['wav_data']))['array']
        for k in range(len(selected)):
            s = selected[k]['start']
            e = selected[k]['end'] + 0.1
            t = selected[k]['text']
            cleaned_t = preprocessing_text(t)
            if not len(cleaned_t):
                continue
                
            audio_path = os.path.join(directory, f'{index}.mp3')
            a = audio[int(s * sr): int(e * sr)]
            torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')
            try:
                y_int = malaya_speech.astype.float_to_int(a)
                frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr, append_ending_trail = False))

                dataset.append(
                    {
                        'file': audio_path,
                        'text': t,
                        'cleaned': cleaned_t,
                        'no': no,
                        'i': i,
                        'k': k,
                        'avg_logprob': selected[k]['avg_logprob'],
                        'no_speech_prob': selected[k]['no_speech_prob'],
                        'vad': float(np.mean([vad(frame) for frame in frames_int])),
                    }
                )
            except:
                pass
            index += 1

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  5.87it/s]


100%|███████████████████████████████████████████| 77/77 [00:10<00:00,  7.49it/s]


 74%|███████████████████████████████▉           | 26/35 [00:03<00:01,  6.25it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████████| 9/9 [00:01<00:00,  5.90it/s]


100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  4.83it/s]


 17%|███████▌                                     | 1/6 [00:00<00:01,  4.94it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  5.91it/s]


...

 30%|█████████████                              | 17/56 [00:04<00:07,  4.96it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 79%|████████████████████████████████▌        | 312/393 [00:51<00:16,  5.02it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 89%|████████████████████████████████████▎    | 329/371 [01:00<00:07,  5.78it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████| 23/23 [00:04<00:00,  5.47it/s]


100%|███████████████████████████████████████████| 34/34 [00:02<00:00, 11.98it/s]


 49%|████████████████████▍                     | 61/125 [00:06<00:10,  6.13it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████| 570/570 [01:43<00:00,  5.49it/s]


 94%|██████████████████████████████████████▋  | 572/607 [01:49<00:06,  5.17it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
len(dataset)

2040228

In [24]:
import json

with open('dataset-whisper-stt-2023-06-19.json', 'w') as fopen:
    json.dump(dataset, fopen)

In [25]:
!ls -lh dataset-whisper-stt-2023-06-19.json

-rw-r--r-- 1 husein husein 592M Jul   2 23:59 dataset-whisper-stt-2023-06-19.json
